In [ ]:
import requests
import pandas as pd

BASE_URL = "https://api.kraken.com/0/public/OHLC"
PAIR = "XXBTZUSD"
INTERVAL = 60  # 1h

# One specific timestamp: 2025-04-20 17:00:00 UTC
since_ts = int(pd.Timestamp("2025-04-20T17:00:00Z").timestamp())

params = {
    "pair": PAIR,
    "interval": INTERVAL,
    "since": since_ts,
}

resp = requests.get(BASE_URL, params=params, timeout=30)
resp.raise_for_status()
payload = resp.json()

print("Request params:", params)
print("Kraken error field:", payload.get("error"))

result = payload["result"]
pair_key = next(k for k in result.keys() if k != "last")
rows = result[pair_key]

# Print raw response rows from this one call
print("\nRaw rows returned:")
print(rows)

# Also print as DataFrame for readability
if rows:
    df = pd.DataFrame(
        rows,
        columns=["timestamp", "open", "high", "low", "close", "vwap", "volume", "trades"],
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s", utc=True)
    print("\nDataFrame view:")
    print(df)
else:
    print("No rows returned.")
